## Function Attributes
---
### Volatility -- IMMUTABLE

In [14]:
CREATE OR REPLACE FUNCTION phi()
    RETURNS DOUBLE PRECISION
LANGUAGE SQL
IMMUTABLE
AS $$
    SELECT (1 + |/ 5) /2;
$$;

SELECT phi();

---
### Volatility -- STABLE

In [15]:
CREATE OR REPLACE FUNCTION get_cust_name(id INTEGER) 
    RETURNS VARCHAR
LANGUAGE SQL
STABLE
AS $$
    SELECT CONCAT_WS(' ', firstname, lastname) 
    FROM customers
    WHERE customerid = id;
$$;

SELECT get_cust_name(42);
UPDATE customers 
SET firstname = 'No Longer', lastname = 'Crazy'
WHERE customerid = 42;
SELECT get_cust_name(42);

---
### Volatility -- VOLATILE

In [16]:
CREATE OR REPLACE FUNCTION get_random_int()
    RETURNS INTEGER
LANGUAGE SQL
VOLATILE
AS $$
    SELECT (random()*10)::INTEGER;
$$;

SELECT get_random_int();

---
### Volatility -- [NOT] LEAKPROOF

In [18]:
CREATE OR REPLACE FUNCTION sometimes_error(flag BOOLEAN) 
    RETURNS VOID
LANGUAGE plpgsql
NOT LEAKPROOF
AS $$
    BEGIN
        IF flag THEN
            RAISE NOTICE 'Parameter is true';
        END IF;
    END;
$$;

SELECT sometimes_error(TRUE);   
SELECT sometimes_error(FALSE);  

---
### Priviledges -- SECURITY DEFINER|INVOKER

In [2]:
CREATE OR REPLACE FUNCTION get_customer(id INT)
    RETURNS SETOF customers
LANGUAGE SQL
SECURITY DEFINER
AS $$
    SELECT * 
    FROM customers
    WHERE customerid = id;
$$;

SELECT get_customer(42);

CREATE USER lucas PASSWORD 'p455w0rd';
REVOKE SELECT ON customers FROM lucas;
GRANT EXECUTE ON FUNCTION get_customer(INT) TO LUCAS;

---
### Handling null input

In [20]:
CREATE OR REPLACE FUNCTION nulls_ok(n INT) RETURNS INT
LANGUAGE SQL 
CALLED ON NULL INPUT
AS $$
    SELECT CASE WHEN n IS NOT NULL THEN n ELSE -42 END;
$$;    

SELECT nulls_ok(42);
SELECT nulls_ok(NULL);

In [21]:
CREATE OR REPLACE FUNCTION no_nulls (n INT) RETURNS INT
LANGUAGE SQL 
RETURNS NULL ON NULL INPUT
AS $$
    SELECT CASE WHEN n IS NOT NULL THEN n ELSE -42 END;
$$;    

SELECT no_nulls(42);
SELECT no_nulls(NULL);

---
### SET search_path

In [24]:
CREATE OR REPLACE FUNCTION restrict_path()
    RETURNS DOUBLE PRECISION
LANGUAGE SQL
SET search_path = pg_temp
AS 'SELECT phi()';

SELECT restrict_path();

---
### Clean up

In [1]:
DROP FUNCTION IF EXISTS get_cust_name(id INTEGER);
DROP FUNCTION IF EXISTS get_random_int();
DROP FUNCTION IF EXISTS sometimes_error(flag BOOLEAN);
DROP FUNCTION IF EXISTS get_customer(id INTEGER);
DROP USER IF EXISTS lucas;
DROP FUNCTION IF EXISTS nulls_ok(n INT);
DROP FUNCTION IF EXISTS no_nulls(n INT);